# Fine tuning classification example

두 스포츠를 구별하기 위해 ADA 분류기를 미세 조정합니다: 야구 그리고 하키.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() # load environment variables from .env file

In [ ]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

 ## Data exploration
 뉴스 그룹 데이터 세트는 sklearn을 사용하여 로드할 수 있습니다. 먼저 데이터 자체를 살펴보겠습니다.:

In [ ]:
print(sports_dataset['data'][0])

In [ ]:
sports_dataset.target_names[sports_dataset['target'][0]]


In [ ]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

야구 범주의 한 샘플은 위에서 볼 수 있습니다. 메일링 리스트로 보내는 메일입니다. 총 1197개의 예가 있으며 두 스포츠 간에 균등하게 분할되어 있음을 관찰할 수 있습니다.

## Data Preparation
프롬프트 및 완료를 위한 열이 있는 pandas 데이터 프레임으로 데이터 세트를 변환합니다. 프롬프트에는 메일링 리스트의 이메일이 포함되며 완료는 하키 또는 야구와 같은 스포츠 이름입니다. 시연 목적과 미세 조정 속도를 위해 300개의 예만 사용합니다. 실제 사용 사례에서는 예제가 많을수록 성능이 향상됩니다.

In [ ]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

야구와 하키는 모두 단일 토큰입니다. 데이터 세트를 jsonl 파일로 저장합니다.

In [ ]:
df.to_json("sport2.jsonl", orient='records', lines=True)

### Data Preparation tool
이제 미세 조정하기 전에 데이터 세트에 대한 몇 가지 개선 사항을 제안하는 데이터 준비 도구를 사용할 수 있습니다. 도구를 실행하기 전에 openai 라이브러리를 업데이트하여 최신 데이터 준비 도구를 사용하고 있는지 확인합니다. 모든 제안을 자동으로 수락하는 `-q`를 추가로 지정합니다.

In [ ]:
!pip install --upgrade openai

In [ ]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

이 도구는 데이터 세트에 대한 몇 가지 개선 사항을 유용하게 제안하고 데이터 세트를 교육 및 검증 세트로 분할합니다.

프롬프트와 완료 사이의 접미사는 입력 텍스트가 중지되었으며 이제 클래스를 예측해야 함을 모델에 알리는 데 필요합니다. 각 예에서 동일한 구분 기호를 사용하기 때문에 모델은 구분 기호 다음에 야구 또는 하키를 예측하기 위한 것임을 학습할 수 있습니다.
대부분의 단어 토큰은 공백 접두사로 토큰화되므로 완성 시 공백 접두사가 유용합니다.
도구는 또한 이것이 분류 작업일 가능성이 있음을 인식하여 데이터 세트를 훈련 및 검증 데이터 세트로 분할할 것을 제안했습니다. 이를 통해 새 데이터에 대한 예상 성능을 쉽게 측정할 수 있습니다.

## Fine-tuning
이 도구는 다음 명령을 실행하여 데이터 세트를 훈련할 것을 제안합니다. 이것은 분류 작업이므로 제공된 유효성 검사 세트의 일반화 성능이 분류 사용 사례에 대해 무엇인지 알고 싶습니다. 이 도구는 분류 메트릭을 계산하기 위해 `--compute_classification_metrics --classification_positive_class " baseball"`을 추가할 것을 제안합니다.

CLI 도구에서 제안된 명령을 간단히 복사할 수 있습니다. 특히 `-m ada`를 추가하여 더 싸고 빠른 ada 모델을 미세 조정합니다. 이는 일반적으로 분류 사용 사례에서 더 느리고 더 비싼 모델과 성능면에서 비교할 수 있습니다.

In [ ]:
!openai api fine_tunes.create -t "sport2_prepared_train.jsonl" -v "sport2_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " baseball" -m ada

모델은 약 10분 안에 성공적으로 학습됩니다. 모델명은 'ada:ft-openai-2021-07-30-12-26-20'으로 추론에 사용할 수 있습니다.

### [Advanced] Results and expected model performance
이제 결과 파일을 다운로드하여 보류된 유효성 검사 세트에서 예상되는 성능을 관찰할 수 있습니다.

In [ ]:
# !openai api fine_tunes.results -i ft-2zaA7qi0rxJduWQpdvOvmGn3 > result.csv
# !openai api fine_tunes.results -i ft-frY2z6vhl3Nh6hBWCqaxFcgF > result.csv

!openai api fine_tunes.results -i ft-12g9fKRq9huNCbvCLBMr1AwL > result.csv

In [ ]:
# results = pd.read_csv('result.csv')
results = pd.read_csv('result.csv')
results[results['classification/accuracy'].notnull()].tail(1)

정확도는 99.6%에 이릅니다. 아래 플롯에서 훈련 실행 중에 유효성 검사 세트의 정확도가 어떻게 증가하는지 확인할 수 있습니다.

In [ ]:
results[results['classification/accuracy'].notnull()]['classification/accuracy'].plot()

## Using the model
이제 모델을 호출하여 예측을 얻을 수 있습니다.

In [ ]:
test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head()

미세 조정 중에 사용한 프롬프트 다음에 동일한 구분 기호를 사용해야 합니다. 이 경우 `\n\n###\n\n`입니다. 우리는 분류와 관련이 있기 때문에 온도가 가능한 한 낮아지기를 원하며 모델의 예측을 결정하기 위해 하나의 토큰 완료만 필요합니다.

In [ ]:
# ft_model = 'ada:ft-openai-2021-07-30-12-26-20'
ft_model = 'ada:ft-personal-2023-04-12-07-55-37'
res = openai.Completion.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0)
res['choices'][0]['text']


로그 확률을 얻기 위해 완료 요청에 logprobs 매개변수를 지정할 수 있습니다.

In [ ]:
res = openai.Completion.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['logprobs']['top_logprobs'][0]

모델이 야구보다 하키를 훨씬 더 많이 예측한다는 것을 알 수 있습니다. 이것이 정확한 예측입니다. log_probs를 요청하면 각 클래스에 대한 예측(로그) 확률을 볼 수 있습니다.

### Generalization
흥미롭게도 미세 조정된 분류기는 매우 다재다능합니다. 다른 메일링 리스트에 대한 이메일에 대한 훈련을 받았음에도 불구하고 트윗을 성공적으로 예측합니다.

In [ ]:
sample_hockey_tweet = """Thank you to the 
@Canes
 and all you amazing Caniacs that have been so supportive! You guys are some of the best fans in the NHL without a doubt! Really excited to start this new chapter in my career with the 
@DetroitRedWings
 !!"""
res = openai.Completion.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['text']

In [ ]:
sample_baseball_tweet="""BREAKING: The Tampa Bay Rays are finalizing a deal to acquire slugger Nelson Cruz from the Minnesota Twins, sources tell ESPN."""
res = openai.Completion.create(model=ft_model, prompt=sample_baseball_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['text']